In [1]:
import sys
import os
import time
import json
import random

import numpy as np
import pandas as pd

import torch as T
import torchvision


/Users/matt/.pyenv/versions/3.10.0/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_videos = os.path.join('.', 'data', 'raw', 'videos')
raw_info = os.path.join('.', 'data', 'raw', 'info')
processed = os.path.join('.', 'data', 'processed')

transform = torchvision.transforms.Compose([
    torchvision.transforms.CenterCrop((720, 720)),
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ConvertImageDtype(T.float)
])

def filename_gen(n_chars=8):
    used = list()
    max_used = 26 * 36**(n_chars-1)
    letters = [chr(k) for k in range(97, 123)]
    numbers = [chr(k) for k in range(48, 58)]
    while len(used) < max_used:
        next_filename = random.choice(letters) + ''.join(
            random.choices(letters+numbers, k=n_chars-1)
        )
        if next_filename not in used:
            used.append(next_filename)
            yield next_filename
        


def process(n_files=None, n_frames=15, freq=2, buffer=10, verbose=False):
    
    total_perf_count = time.perf_counter()
    
    if n_files is None:
        n_files = -1
    file_list = [
        os.path.splitext(filename)[0]
        for filename in os.listdir(raw_videos)[:n_files]
        if filename != '.DS_Store'
    ]
    
    labels_list = list()
    
    for num, filename in enumerate(file_list):
    
        inner_perf_count = time.perf_counter()
        
        vid_path = os.path.join(raw_videos, filename + '.mov')
        info_path = os.path.join(raw_info, filename + '.json')
        
        with open(info_path) as f:
            json_data = json.load(f)
        
        segments = list()
        
        if any(reading['accuracy'] == 10.0 for reading in json_data['locations']):
            
            vid, _, metadata = torchvision.io.read_video(vid_path)
            vid = T.permute(vid, (0, 3, 2, 1))

            vid_frame_rate = metadata['video_fps']
            vid_start_time = json_data['startTime']
            next_possible_timestamp = vid_start_time

            for reading in json_data['locations']:
                if reading['timestamp'] >= next_possible_timestamp and reading['accuracy'] == 10.0:

                    index_timestamp = round(
                        (reading['timestamp'] - vid_start_time) / 1000 * vid_frame_rate
                    )
                    index_end = index_timestamp + freq
                    index_start = index_end - n_frames * freq
                    if index_start < 0 or index_end >= len(vid):
                        continue

                    frames = vid[index_start: index_end: freq, :, :, :]
                    frames = transform(frames)

                    segments.append((frames, reading['speed']))
                    next_possible_timestamp = reading['timestamp'] + buffer * 1000
                    
        if segments:

            for new_filename, (sample, label) in zip(filename_gen(), segments):
                T.save(
                    sample,
                    os.path.join(processed, 'samples', new_filename)
                )
                labels_list.append((new_filename, label))
        
        if verbose:
            if num == 0:
                print('Finished loading file' + (len(file_list) > 1) * 's' + ':')
            print(
                f'{time.perf_counter() - inner_perf_count:>8.3f}s :: ' +
                f'{filename} -> {len(segments)} examples. ' +
                f'({num+1}/{len(file_list)})'
            )
        
    labels_df = pd.DataFrame(labels_list, columns=['filename', 'label'])
    labels_df.to_csv(os.path.join(processed, 'labels.csv'))
    
    if verbose:
        elapsed = int(time.perf_counter() - total_perf_count)
        print(
            f'Done. Generated {len(segments)} examples in ' +
            f'{elapsed // 3600}:{elapsed % 3600 // 60:02}:{elapsed % 60:02}.'
        )


In [3]:
process(verbose=True)

Finished loading files:
   8.406s :: 0582d8e4-000aa194 -> 4 examples. (1/2999)
  14.334s :: 0465ebb7-4da7a012 -> 4 examples. (2/2999)
  12.385s :: 0104a3f0-9294818d -> 4 examples. (3/2999)
   0.008s :: 02f208e7-904032da -> 0 examples. (4/2999)
  11.512s :: 0505c36e-579b416c -> 4 examples. (5/2999)
  12.245s :: 07116ae0-e9b4fa4a -> 4 examples. (6/2999)
   0.013s :: 014070bf-4de0af14 -> 0 examples. (7/2999)
  10.724s :: 03d75135-9aa0cdd3 -> 4 examples. (8/2999)
  10.565s :: 041bf7e5-c5703bb5 -> 4 examples. (9/2999)
  13.443s :: 02480aee-cb6bb3a5 -> 4 examples. (10/2999)
  14.810s :: 02494b7b-589045ab -> 2 examples. (11/2999)
  13.395s :: 024ac038-8a8b481c -> 4 examples. (12/2999)
  13.124s :: 04e47e69-f7459753 -> 4 examples. (13/2999)
   0.014s :: 02514ff2-b8e3551c -> 0 examples. (14/2999)
  12.811s :: 006a4209-286a5664 -> 4 examples. (15/2999)
  14.761s :: 0725b3e1-9c9078cb -> 4 examples. (16/2999)
  16.053s :: 06d5025c-bb0edff4 -> 4 examples. (17/2999)
   0.056s :: 031df6ea-15e2b64a ->

  10.863s :: 052d8d40-89ac0197 -> 4 examples. (147/2999)
  11.371s :: 000f157f-30b30f5e -> 4 examples. (148/2999)
  13.429s :: 03b3c151-f37ef8b1 -> 4 examples. (149/2999)
  10.045s :: 06086d96-9282b1aa -> 4 examples. (150/2999)
  10.554s :: 01460ec4-41af799a -> 1 examples. (151/2999)
   0.014s :: 0701abd4-d9c2e1f4 -> 0 examples. (152/2999)
  13.167s :: 01bc8687-ad6aee3c -> 4 examples. (153/2999)
  11.092s :: 056db1a0-b6a1bb41 -> 4 examples. (154/2999)
  10.996s :: 025ce5d9-afedd5c5 -> 4 examples. (155/2999)
  11.640s :: 03978fc8-b1fe3a3f -> 4 examples. (156/2999)
   0.010s :: 0362f5f6-8f12c274 -> 0 examples. (157/2999)
  10.452s :: 0315121e-d61f498f -> 4 examples. (158/2999)
  13.555s :: 02c0b2f1-6a191bd8 -> 4 examples. (159/2999)
  12.535s :: 01a78dfd-ba2fca5e -> 4 examples. (160/2999)
  12.305s :: 028b5d16-af6a6275 -> 4 examples. (161/2999)
   4.993s :: 06ac7da9-81ae9c1c -> 2 examples. (162/2999)
   9.347s :: 01b4e4b9-fc7a5264 -> 4 examples. (163/2999)
   9.115s :: 03c239e4-2edeb109 

   4.865s :: 05f073fb-c158d772 -> 2 examples. (291/2999)
   0.016s :: 077db2be-4784f73c -> 0 examples. (292/2999)
   0.009s :: 030003dd-480d0868 -> 0 examples. (293/2999)
   9.411s :: 05c26f8e-455c67dc -> 4 examples. (294/2999)
  12.663s :: 0391b282-79dba56d -> 4 examples. (295/2999)
  12.386s :: 00c17a92-d4803287 -> 4 examples. (296/2999)
  11.749s :: 03105f99-28a17a6c -> 4 examples. (297/2999)
  12.387s :: 00417c23-220bbc98 -> 4 examples. (298/2999)
  11.779s :: 043c7d97-e7694565 -> 4 examples. (299/2999)
  11.371s :: 05e4d323-6779ccbc -> 4 examples. (300/2999)
  11.027s :: 07be42b3-74ca93ff -> 4 examples. (301/2999)
  11.409s :: 03bf498b-044cb72c -> 4 examples. (302/2999)
  13.480s :: 03aa4d2b-a9a49f02 -> 4 examples. (303/2999)
  12.008s :: 01041028-15488454 -> 4 examples. (304/2999)
  10.406s :: 05b4fcee-a4af791b -> 4 examples. (305/2999)
  10.461s :: 074f53a8-085ed8fa -> 2 examples. (306/2999)
  12.021s :: 048e0c14-37c19b65 -> 4 examples. (307/2999)
  13.158s :: 0514a4f5-d88db8b5 

  11.470s :: 05c1a69f-6a94dbe0 -> 2 examples. (435/2999)
  10.852s :: 0573beee-2e0c3c4d -> 4 examples. (436/2999)
  12.490s :: 061015c1-a0d2ddc5 -> 4 examples. (437/2999)
  12.459s :: 027e72be-8b4f222c -> 4 examples. (438/2999)
  12.528s :: 04ede4bd-c03f27b1 -> 4 examples. (439/2999)
   0.015s :: 05986b6d-fb96be0d -> 0 examples. (440/2999)
  12.213s :: 05d4dd7d-7eb9680a -> 4 examples. (441/2999)
  12.344s :: 0429dca3-163013e9 -> 4 examples. (442/2999)
  10.917s :: 06031175-b4b8a11e -> 4 examples. (443/2999)
  13.612s :: 00cef86b-204ea619 -> 4 examples. (444/2999)
  11.011s :: 05110707-50c2b28c -> 4 examples. (445/2999)
  13.127s :: 050b5f87-684191b9 -> 4 examples. (446/2999)
  10.837s :: 0598d1c5-932bc43d -> 4 examples. (447/2999)
  11.353s :: 030e8b7a-4dab7745 -> 4 examples. (448/2999)
   0.015s :: 0612e307-beec11ce -> 0 examples. (449/2999)
   0.009s :: 03db7788-3dc1f477 -> 0 examples. (450/2999)
   0.010s :: 010799f0-4d81c585 -> 0 examples. (451/2999)
   4.753s :: 05c1c47a-cb817b64 

  10.659s :: 02c01e62-c53e91a0 -> 4 examples. (579/2999)
   9.926s :: 03d93dd6-990f83bb -> 4 examples. (580/2999)
   9.409s :: 04d49b8a-f0e342cb -> 4 examples. (581/2999)
  11.460s :: 00787a90-b350f376 -> 4 examples. (582/2999)
  10.752s :: 013162b0-ddbabae8 -> 4 examples. (583/2999)
  11.852s :: 047bb958-7280e29f -> 4 examples. (584/2999)
   9.744s :: 0128cdde-ce2cb20b -> 4 examples. (585/2999)
   4.450s :: 07b1efc6-4e6ba1fd -> 2 examples. (586/2999)
   0.013s :: 060fcfc7-c5c91f05 -> 0 examples. (587/2999)
   8.183s :: 006d6f8e-4759e8c1 -> 4 examples. (588/2999)
  11.237s :: 07398e6d-7b0dc522 -> 4 examples. (589/2999)
   0.014s :: 04dc483a-f595d9f1 -> 0 examples. (590/2999)
   9.429s :: 0549f27d-fe32da2d -> 4 examples. (591/2999)
   9.664s :: 04629c39-69d7c08c -> 4 examples. (592/2999)
   0.009s :: 0048f391-2c5344eb -> 0 examples. (593/2999)
  10.546s :: 01a8723d-b7f960ce -> 4 examples. (594/2999)
   9.534s :: 069365c1-704b7a9b -> 4 examples. (595/2999)
  11.756s :: 0798a3a8-c4501e0f 

...

   8.567s :: 04200e90-4a4c631e -> 4 examples. (2290/2999)
  10.452s :: 0492b183-8ce155d4 -> 4 examples. (2291/2999)
   0.012s :: 067e1bbf-39b15383 -> 0 examples. (2292/2999)
   9.833s :: 04c1910d-7df8b6d0 -> 4 examples. (2293/2999)
  10.776s :: 0521fd41-40219907 -> 4 examples. (2294/2999)
  11.143s :: 02c26e9d-e59d7856 -> 4 examples. (2295/2999)
   9.258s :: 0337fd00-e231ff51 -> 4 examples. (2296/2999)
  10.854s :: 076399f2-6c2cdfd0 -> 4 examples. (2297/2999)
   9.758s :: 03a5976c-6d4c6239 -> 4 examples. (2298/2999)
   0.010s :: 006fdb67-f4820206 -> 0 examples. (2299/2999)
   8.620s :: 05584a52-730a88df -> 4 examples. (2300/2999)
   8.821s :: 0096bcca-81ffedfc -> 3 examples. (2301/2999)
   8.766s :: 047cf4fc-34ad5b08 -> 4 examples. (2302/2999)
   8.880s :: 0631bd6c-14480fea -> 4 examples. (2303/2999)
   8.816s :: 03343856-ebf78b5f -> 4 examples. (2304/2999)
   9.981s :: 04be39da-0d1f5c97 -> 4 examples. (2305/2999)
  10.191s :: 06fb50bb-6d44b337 -> 4 examples. (2306/2999)
   0.009s :: 0

   4.484s :: 0531d2f7-1feee72c -> 2 examples. (2432/2999)
   0.009s :: 01fa77d0-e86c6400 -> 0 examples. (2433/2999)
   8.681s :: 06e80a6c-d721231e -> 4 examples. (2434/2999)
   9.309s :: 00091078-c1d32eea -> 4 examples. (2435/2999)
  10.556s :: 0229e497-f4eed706 -> 0 examples. (2436/2999)
   8.853s :: 0679098c-550ac091 -> 4 examples. (2437/2999)
   8.875s :: 0245bba2-d34755ef -> 3 examples. (2438/2999)
   8.542s :: 041ca12a-3e6c3c5a -> 4 examples. (2439/2999)
   9.494s :: 076399f2-3cc0e470 -> 4 examples. (2440/2999)
   5.422s :: 072ca47b-0cb52c7f -> 3 examples. (2441/2999)
   8.698s :: 035cb512-2f9fa97d -> 4 examples. (2442/2999)
  10.886s :: 01128d0a-031e0a61 -> 4 examples. (2443/2999)
   9.643s :: 05e52595-ee5a9ef4 -> 3 examples. (2444/2999)
  10.820s :: 04299f0f-826f428b -> 4 examples. (2445/2999)
   8.975s :: 03d0bae7-9627a648 -> 4 examples. (2446/2999)
   0.012s :: 07707e5b-2c3664c4 -> 0 examples. (2447/2999)
   0.008s :: 007aeb45-eef41701 -> 0 examples. (2448/2999)
  10.210s :: 0

   8.607s :: 001c5339-13a07470 -> 4 examples. (2574/2999)
   0.011s :: 03866c0e-2a861ef2 -> 0 examples. (2575/2999)
   8.018s :: 014e76d7-afb481a3 -> 4 examples. (2576/2999)
   8.654s :: 0390cdb2-1f193811 -> 4 examples. (2577/2999)
   8.512s :: 031675b4-5ca668c9 -> 1 examples. (2578/2999)
   8.800s :: 0226f407-491ed238 -> 4 examples. (2579/2999)
   0.009s :: 009b6a87-b15675e2 -> 0 examples. (2580/2999)
   9.452s :: 02778d17-efa6d1ee -> 4 examples. (2581/2999)
   8.246s :: 04c49cc4-f994251e -> 1 examples. (2582/2999)
   8.313s :: 05e11010-54bec48b -> 4 examples. (2583/2999)
   8.578s :: 0065fe83-c80cf6c3 -> 4 examples. (2584/2999)
   8.722s :: 006dd004-34630e8d -> 4 examples. (2585/2999)
   8.868s :: 0149d509-819eec9c -> 4 examples. (2586/2999)
   8.646s :: 068fa72c-d5f1b0d9 -> 4 examples. (2587/2999)
   9.891s :: 0245bba2-091b06af -> 4 examples. (2588/2999)
   7.933s :: 03031fa8-c168834a -> 4 examples. (2589/2999)
   8.892s :: 0334d6ba-b42d12c0 -> 4 examples. (2590/2999)
   8.683s :: 0

   9.038s :: 066f1d1e-7e16e205 -> 4 examples. (2716/2999)
   8.800s :: 03c28a8d-9fd776c9 -> 1 examples. (2717/2999)
   8.478s :: 07b5edf7-e4c83fc5 -> 1 examples. (2718/2999)
   0.011s :: 034fbacd-2017ee15 -> 0 examples. (2719/2999)
   8.721s :: 02a39657-a59abfc7 -> 4 examples. (2720/2999)
   4.276s :: 02e06911-cb3fd71e -> 2 examples. (2721/2999)
   0.008s :: 00ccf2e8-ac055be6 -> 0 examples. (2722/2999)
   8.451s :: 047671d6-83485ddb -> 4 examples. (2723/2999)
   8.603s :: 05b4fcee-74ffe788 -> 4 examples. (2724/2999)
   9.308s :: 043026cb-a30ed3f0 -> 4 examples. (2725/2999)
   9.304s :: 031637c4-b28f871a -> 4 examples. (2726/2999)
   0.009s :: 0102614a-75ae85cf -> 0 examples. (2727/2999)
   0.007s :: 0325b131-11b7d3a0 -> 0 examples. (2728/2999)
   9.312s :: 01c9256f-b4754064 -> 4 examples. (2729/2999)
  11.463s :: 06297d68-dce282b3 -> 4 examples. (2730/2999)
   4.742s :: 01440a35-e0b2f4a2 -> 2 examples. (2731/2999)
   0.010s :: 0711ab13-ff3633ab -> 0 examples. (2732/2999)
   0.009s :: 0

  10.689s :: 02cc2722-171d6b7f -> 4 examples. (2858/2999)
   8.719s :: 045fbeb8-3b795ebb -> 4 examples. (2859/2999)
   8.423s :: 00ce8219-12c6d905 -> 4 examples. (2860/2999)
   4.089s :: 033dea13-5a8114fe -> 2 examples. (2861/2999)
   8.692s :: 06f9786e-16c538fd -> 4 examples. (2862/2999)
   8.649s :: 06d1e27a-417be915 -> 4 examples. (2863/2999)
   8.824s :: 04180105-d7aca785 -> 4 examples. (2864/2999)
   8.509s :: 06c3126b-de9504ce -> 4 examples. (2865/2999)
   0.009s :: 01e87cf1-25268f60 -> 0 examples. (2866/2999)
   8.689s :: 076260e7-fb7ec9a6 -> 1 examples. (2867/2999)
   9.111s :: 0168e22b-81d10cb6 -> 4 examples. (2868/2999)
   9.620s :: 060c3ea8-e0626114 -> 4 examples. (2869/2999)
  10.171s :: 0045e757-02530231 -> 4 examples. (2870/2999)
   0.016s :: 039f79c0-66d9d6ca -> 0 examples. (2871/2999)
   8.779s :: 057abd7e-e2b7aad8 -> 4 examples. (2872/2999)
   8.662s :: 0096bcca-fc7d6d3c -> 4 examples. (2873/2999)
  11.803s :: 03c239e4-9d278671 -> 4 examples. (2874/2999)
   0.049s :: 0

In [36]:
mean_list = [[] for _ in range(3)]
std_list = [[] for _ in range(3)]

mean_sum = T.zeros(3)
square_sum = T.zeros(3)
num_samples = 0

for filename in os.listdir(os.path.join(processed, 'samples')):
    if filename != '.DS_Store':
        sample = T.load(os.path.join(processed, 'samples', filename))
        mean_sum += T.mean(sample, dim=[0, 2, 3])
        square_sum += T.mean(sample**2, dim=[0, 2, 3])
        num_samples += 1

dataset_mean = mean_sum / num_samples
dataset_std = (square_sum / num_samples - dataset_mean ** 2) ** 0.5

In [39]:
normalize = torchvision.transforms.Normalize(dataset_mean, dataset_std)
for filename in os.listdir(os.path.join(processed, 'samples')):
    if filename != '.DS_Store':
        sample = T.load(os.path.join(processed, 'samples', filename))
        sample = normalize(sample)
        T.save(sample, os.path.join(processed, 'samples', filename))